In [1]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.6 MB/s eta 0:00:00m eta 0:00:010:00:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 5.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 5.6 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.5 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 5.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.7 MB/s eta 0:00:006.2 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 5.6 MB/s eta 0:00:00m eta 0:00:010:01:01


  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.1
    Uninstalling protobuf-3.20.1:
      Successfully uninstalled protobuf-3.20.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.10.0
    Uninstalling tensorboard-2.10.0:
      Successfully uninstalled tensorboard-2.10.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [2]:
!pip install keras-image-helper

In [4]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [6]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [7]:
stub

In [9]:
from keras_image_helper import create_preprocessor

In [10]:
preprocessor = create_preprocessor('xception', target_size=(299,299))

In [13]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [14]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape = data.shape)

In [15]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'
pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))


In [18]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [19]:
pb_response

outputs {
  key: "dense_7"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 10
      }
    }
    float_val: -1.8682903051376343
    float_val: -4.761245250701904
    float_val: -2.316983461380005
    float_val: -1.0625708103179932
    float_val: 9.887161254882812
    float_val: -2.8124334812164307
    float_val: -3.6662826538085938
    float_val: 3.200361728668213
    float_val: -2.6023378372192383
    float_val: -4.835046291351318
  }
}
model_spec {
  name: "clothing-model"
  version {
    value: 1
  }
  signature_name: "serving_default"
}

In [24]:
preds = pb_response.outputs['dense_7'].float_val

In [25]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [26]:
dict(zip(classes, preds))

{'dress': -1.8682903051376343,
 'hat': -4.761245250701904,
 'longsleeve': -2.316983461380005,
 'outwear': -1.0625708103179932,
 'pants': 9.887161254882812,
 'shirt': -2.8124334812164307,
 'shoes': -3.6662826538085938,
 'shorts': 3.200361728668213,
 'skirt': -2.6023378372192383,
 't-shirt': -4.835046291351318}